In [ ]:
!pip install virtualenv
# Create a new virtual environment
!virtualenv env_train_main
# Activate the virtual environment
!source env_train_main/bin/activate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 29.5 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 400ms
  creator CPython3Posix(dest=/content/env_train_main, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
#!pip install transformers datasets
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
df = pd.DataFrame(dataset['train'])

In [ ]:

print(df.head())

                                            sentence  label
0  According to Gran , the company has no plans t...      1
1  For the last quarter of 2010 , Componenta 's n...      2
2  In the third quarter of 2010 , net sales incre...      2
3  Operating profit rose to EUR 13.1 mn from EUR ...      2
4  Operating profit totalled EUR 21.1 mn , up fro...      2


In [ ]:
# Data Exploration

print("\nSummary statistics:")
print(df.describe(include='all'))

print("\nChecking for missing values:")
print(df.isnull().sum())

print("\nValue counts for sentiment labels:")
print(df['label'].value_counts())


Summary statistics:
                                                 sentence        label
count                                                2264  2264.000000
unique                                               2259          NaN
top     SSH Communications Security Corporation is hea...          NaN
freq                                                    2          NaN
mean                                                  NaN     1.117933
std                                                   NaN     0.609801
min                                                   NaN     0.000000
25%                                                   NaN     1.000000
50%                                                   NaN     1.000000
75%                                                   NaN     2.000000
max                                                   NaN     2.000000

Checking for missing values:
sentence    0
label       0
dtype: int64

Value counts for sentiment labels:
1    1391
2     570


In [ ]:
# Function to compute metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [ ]:
# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

In [ ]:
# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
# Splitting the dataset into training and testing sets
train_test_split = tokenized_dataset["train"].train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [ ]:
# Load pre-trained RoBERTa model with a classification head
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=30,
)

In [ ]:
# Initialize Trainer with metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.226300,0.187237,0.949227,0.950051,0.953550,0.949227
2,0.112200,0.112456,0.977925,0.978046,0.978513,0.977925
3,0.057500,0.145863,0.971302,0.971565,0.972089,0.971302
4,0.027500,0.122254,0.982340,0.982338,0.982567,0.982340
5,0.025600,0.139759,0.980132,0.980217,0.980430,0.980132


TrainOutput(global_step=570, training_loss=0.11973016560404447, metrics={'train_runtime': 298.2681, 'train_samples_per_second': 30.359, 'train_steps_per_second': 1.911, 'total_flos': 2382491997527040.0, 'train_loss': 0.11973016560404447, 'epoch': 5.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate(test_dataset)
print(results)

{'eval_loss': 0.11245623230934143, 'eval_accuracy': 0.977924944812362, 'eval_f1': 0.9780455285648365, 'eval_precision': 0.9785125054751755, 'eval_recall': 0.977924944812362, 'eval_runtime': 4.4157, 'eval_samples_per_second': 102.587, 'eval_steps_per_second': 6.567, 'epoch': 5.0}


In [ ]:
torch.save(model.state_dict(), 'model.pth')